In [6]:
# Importing needed libraries
import os
import pandas as pd
import cv2
import mediapipe as mp
from mediapipe.tasks.python import vision
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.src.callbacks import ReduceLROnPlateau
from keras.src.models import Sequential
from keras.src.layers import Conv2D, BatchNormalization, MaxPooling2D, Dense, Flatten, Dropout, Input
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

In [5]:
# Setting the path to the data
path = os.path.abspath('..')

train_path = os.path.join(path, 'dataset', 'ASL_Dataset', 'Train')
test_path = os.path.join(path, 'dataset', 'ASL_Dataset', 'Test')
model_path = os.path.join(path, 'models' ,'hand_landmarker.task')

In [33]:
# Label map
label_map = {chr(i): i - ord('A') for i in range(ord('A'), ord('Z') + 1)}
label_map['Nothing'] = 26
label_map['Space'] = 27
# Utils functions
def set_to_csv(input, output):
    # Initialize an empty list to store data
    data = []
    
    # Iterate through each subfolder in the training path
    for label_folder in sorted(os.listdir(input)):
        folder_path = os.path.join(input, label_folder)
        if os.path.isdir(folder_path) and label_folder in label_map:
            label = label_map[label_folder]
            # Iterate through each image file in the folder
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                # Read the image
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                # Resize to 50x50 pixels
                resized_image = cv2.resize(image, (50, 50))
                # Flatten the image to a 1D array
                flattened_image = resized_image.flatten()
                # Append the label and flattened image to the data
                data.append([label] + flattened_image.tolist())
    
    # Create a DataFrame with the data
    columns = ['Label'] + [f'Pixel_{i}' for i in range(50 * 50)]
    df = pd.DataFrame(data, columns=columns)
    
    # Save the DataFrame to a CSV file
    df.to_csv(output, index=False)
    
    print("CSV file with label mappings created successfully!")


BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the video mode:
options = HandLandmarkerOptions(
    num_hands=1,
    min_hand_detection_confidence=0.6,
    min_hand_presence_confidence=0.6,
    min_tracking_confidence=0.5,
    base_options=BaseOptions(model_asset_path='../models/hand_landmarker.task'),running_mode=VisionRunningMode.IMAGE)
detector = vision.HandLandmarker.create_from_options(options)

def landmarks_to_csv(input, output):
    # Initialize an empty list to store data
    data = []
    
    # Iterate through each subfolder in the training path
    for label_folder in sorted(os.listdir(input)):
        folder_path = os.path.join(input, label_folder)
        print(folder_path)
        if os.path.isdir(folder_path) and label_folder in label_map:
            label = label_map[label_folder]
            # Iterate through each image file in the folder
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                # Read the image
                image = cv2.imread(image_path)
                # Convert the image to RGB as required by the model
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    
                # Process the image to detect hand landmarks
                result = detector.detect(mp_image)
    
                if result.handedness:  # Check if hands are detected
                    # Extract landmarks of the first detected hand
                    landmarks = result.hand_landmarks[0]
                    # Collect (x, y) coordinates of each landmark
                    landmark_list = []
                    for lm in landmarks:
                        landmark_list.extend([lm.x, lm.y])
                    # Append the label and landmarks to the data
                    data.append([label] + landmark_list)
                else:
                    print(f"No hand landmarks detected in {folder_path}/{image_file}")
    
    # Define column names for the CSV
    num_landmarks = 21  # MediaPipe detects 21 landmarks per hand
    columns = ['Label'] + [f'X{i+1}' for i in range(num_landmarks)] + [f'Y{i+1}' for i in range(num_landmarks)]


    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output, index=False)
    
    print("CSV file with hand landmarks created successfully!")


I0000 00:00:1731866181.263183   12632 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1731866181.266092   13703 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: AMD Radeon Graphics (radeonsi, gfx1103_r1, LLVM 17.0.6, DRM 3.57, 6.8.0-48-generic)
W0000 00:00:1731866181.296512   13709 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731866181.305758   13705 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [29]:
# Create CSV files for training and testing data
set_to_csv(train_path, '../dataset/train.csv')
set_to_csv(test_path, '../dataset/test.csv')

KeyboardInterrupt: 

In [34]:
landmarks_to_csv(train_path, '../dataset/train_landmarks.csv')
landmarks_to_csv(test_path, '../dataset/test_landmarks.csv')

/home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/A
/home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/B
/home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/C592.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/C593.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/C611.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/2617.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/C2384.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/C2375.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/2612.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_Dataset/Train/C/C601.jpg
No hand landmarks detected in /home/llaczko/Workspace/SLR/dataset/ASL_

KeyboardInterrupt: 